In [ ]:
#动手练习：手写数字识别

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset
import torch.utils.data.dataloader as dataloader
from torch.nn.parameter import Parameter
import torchvision.transforms as transforms

In [4]:
train_set = torchvision.datasets.MNIST(root="./",train=True,transform=transforms.ToTensor(),download=False)
train_set_A=Subset(train_set,range(0,1000))
train_set_B=Subset(train_set,range(1000,2000))
train_set_C=Subset(train_set,range(2000,3000))#使用subset函数分成3个数据集
train_loader_A = dataloader.DataLoader(dataset=train_set_A,batch_size=1000,shuffle=False)
train_loader_B = dataloader.DataLoader(dataset=train_set_B,batch_size=1000,shuffle=False)
train_loader_C = dataloader.DataLoader(dataset=train_set_C,batch_size=1000,shuffle=False)
test_set = torchvision.datasets.MNIST(root="./",train=False,transform=transforms.ToTensor(),download=False)
test_set=Subset(test_set,range(0,2000))
test_loader = dataloader.DataLoader(dataset=test_set,shuffle=True)

In [5]:
def train_and_test_1(train_loader,test_loader):
    class NeuralNet(nn.Module):
        def __init__(self, input_num, hidden_num, output_num):
            super(NeuralNet, self).__init__()
            self.fc1 = nn.Linear(input_num, hidden_num)  #服从正态分布
            self.fc2 = nn.Linear(hidden_num, output_num)
            nn.init.normal_(self.fc1.weight)# 将fc1的权重初始化为标准正态分布（均值为0，标准差为1）
            nn.init.normal_(self.fc2.weight)
            nn.init.constant_(self.fc1.bias, val=0)  #初始化bias为0
            nn.init.constant_(self.fc2.bias, val=0)  # 将fc1的偏置项全部初始化为0
            self.relu = nn.ReLU()   #Relu激活函数

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu(x)  
            y = self.fc2(x)
            return y

    epoches = 50  #迭代50轮
    lr = 0.01     #学习率
    input_num = 784
    hidden_num = 12
    output_num = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = NeuralNet(input_num, hidden_num, output_num)
    model.to(device)
    loss_func = nn.CrossEntropyLoss()   #损失函数的类型：交叉熵损失函数
    optimizer = optim.Adam(model.parameters(), lr=lr)  #使用Adam优化器
    for epoch in range(epoches):
        flag = 0
        for images, labels in train_loader:
            images = images.reshape(-1, 28 * 28).to(device)
            labels = labels.to(device)
            output = model(images)

            loss = loss_func(output, labels)
            optimizer.zero_grad()
            loss.backward()   #误差反向传播，计算参数更新值
            optimizer.step()  #将参数更新值施加到net的parameters上
            
            flag += 1
    
    params = list(model.named_parameters())  #获取模型参数
    
    #测试模型，评估准确率
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        output = model(images)
        values, predicte = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicte == labels).sum().item()
    print("The accuracy of total {} images: {}%".format(total, 100 * correct / total))
    return params

In [7]:
def train_and_test_2(train_loader,test_loader,com_para_fc1,com_para_fc2):
    class NeuralNet(nn.Module):
        def __init__(self, input_num, hidden_num, output_num,com_para_fc1,com_para_fc2):
            super(NeuralNet, self).__init__()
            self.fc1 = nn.Linear(input_num, hidden_num)
            self.fc2 = nn.Linear(hidden_num, output_num)
            self.fc1.weight=Parameter(com_para_fc1)
            self.fc2.weight=Parameter(com_para_fc2)
            nn.init.constant_(self.fc1.bias, val=0)
            nn.init.constant_(self.fc2.bias, val=0)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu(x)
            y = self.fc2(x)
            return y

    epoches = 20
    lr = 0.01
    input_num = 784
    hidden_num = 12
    output_num = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = NeuralNet(input_num, hidden_num, output_num,com_para_fc1,com_para_fc2)
    model.to(device)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epoches):
        flag = 0
        for images, labels in train_loader:
            images = images.reshape(-1, 28 * 28).to(device)
            labels = labels.to(device)
            output = model(images)

            loss = loss_func(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            flag += 1
    params = list(model.named_parameters())

    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        output = model(images)
        values, predicte = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicte == labels).sum().item()
    print("The accuracy of total {} images: {}%".format(total, 100 * correct / total))
    return params

In [9]:
def combine_params(para_A,para_B,para_C):
    fc1_wA=para_A[0][1].data
    fc1_wB=para_B[0][1].data
    fc1_wC=para_C[0][1].data
  
    fc2_wA=para_A[2][1].data
    fc2_wB=para_B[2][1].data
    fc2_wC=para_C[2][1].data

    com_para_fc1=(fc1_wA+fc1_wB+fc1_wC)/3
    com_para_fc2=(fc2_wA+fc2_wB+fc2_wC)/3
    return com_para_fc1,com_para_fc2

In [10]:
para_A=train_and_test_1(train_loader_A,test_loader)
para_B=train_and_test_1(train_loader_B,test_loader)
para_C=train_and_test_1(train_loader_C,test_loader)
for i in range(10): 
    print("The {} round to be federated!!!".format(i+1))
    com_para_fc1,com_para_fc2=combine_params(para_A,para_B,para_C)
    para_A=train_and_test_2(train_loader_A,test_loader,com_para_fc1,com_para_fc2)
    para_B=train_and_test_2(train_loader_B,test_loader,com_para_fc1,com_para_fc2)
    para_C=train_and_test_2(train_loader_C,test_loader,com_para_fc1,com_para_fc2)

The accuracy of total 2000 images: 12.25%
The accuracy of total 2000 images: 33.55%
The accuracy of total 2000 images: 25.8%
The 1 round to be federated!!!
The accuracy of total 2000 images: 23.1%
The accuracy of total 2000 images: 39.6%
The accuracy of total 2000 images: 48.2%
The 2 round to be federated!!!
The accuracy of total 2000 images: 52.6%
The accuracy of total 2000 images: 56.45%
The accuracy of total 2000 images: 61.85%
The 3 round to be federated!!!
The accuracy of total 2000 images: 62.8%
The accuracy of total 2000 images: 66.7%
The accuracy of total 2000 images: 66.95%
The 4 round to be federated!!!
The accuracy of total 2000 images: 68.6%
The accuracy of total 2000 images: 73.95%
The accuracy of total 2000 images: 74.3%
The 5 round to be federated!!!
The accuracy of total 2000 images: 75.4%
The accuracy of total 2000 images: 73.1%
The accuracy of total 2000 images: 71.6%
The 6 round to be federated!!!
The accuracy of total 2000 images: 71.45%
The accuracy of total 2000 i